In [5]:
import numpy as np
import plotly.graph_objects as go

# AQI Calculation for PM2.5 (EPA Standard)
def calculate_aqi(pm25):
    breakpoints = [
        (0.0, 12.0, 0, 50),
        (12.1, 35.4, 51, 100),
        (35.5, 55.4, 101, 150),
        (55.5, 150.4, 151, 200),
        (150.5, 250.4, 201, 300),
        (250.5, 500.4, 301, 500)
    ]
    labels = ['Good', 'Moderate', 'Unhealthy for Sensitive Groups', 'Unhealthy', 'Very Unhealthy', 'Hazardous']

    for c_low, c_high, aqi_low, aqi_high in breakpoints:
        if c_low <= pm25 <= c_high:
            aqi = ((aqi_high - aqi_low) / (c_high - c_low)) * (pm25 - c_low) + aqi_low
            return labels[breakpoints.index((c_low, c_high, aqi_low, aqi_high))], round(aqi)
    if pm25 > 500.4:
        return 'Hazardous', 501
    return 'Good', 0

# Health Recommendations with Emoji based on AQI
def get_health_recommendation(aqi):
    if 0 <= aqi <= 50:
        return "Air quality is good. No precautions needed.", "😊"
    elif 51 <= aqi <= 100:
        return "Air quality is acceptable. Sensitive groups may take care.", "🙂"
    elif 101 <= aqi <= 150:
        return "Reduce outdoor activities for sensitive groups.", "😷"
    elif 151 <= aqi <= 200:
        return "Wear a mask and limit outdoor exposure.", "😷"
    elif 201 <= aqi <= 300:
        return "Stay indoors and use air purifiers.", "🏠"
    else:
        return "Avoid all outdoor activities; wear N95 masks if outside.", "⚠️"

# Interactive Air Quality Map
def plot_interactive_map(C_pm25, aqi_label, nx=100, ny=80, dx=80, dy=87.5):
    fig = go.Figure(data=go.Heatmap(
        z=C_pm25,
        x=np.arange(0, nx*dx, dx),
        y=np.arange(0, ny*dy, dy),
        colorscale='Viridis',
        colorbar=dict(title='PM2.5 (µg/m³)')
    ))
    fig.update_layout(
        title=f'Air Quality Map: {aqi_label}',
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        xaxis=dict(tickformat='k', tickangle=0),  # Thousand format (e.g., 2k, 4k)
        yaxis=dict(tickformat='.0f'),  # Integer labels
        autosize=False,
        width=800,
        height=600
    )
    fig.show()

# Get user input with validation
def get_user_input():
    params = [
        ('PM2.5', 'Enter PM2.5 in µg/m³: '),
        ('PM10', 'Enter PM10 in µg/m³: '),
        ('NO2', 'Enter NO2 in ppb: '),
        ('CO', 'Enter CO in ppm: '),
        ('SO2', 'Enter SO2 in ppb: '),
        ('O3', 'Enter O3 in ppb: '),
        ('Temperature', 'Enter Temperature in Celsius: '),
        ('Humidity', 'Enter Humidity in %: '),
        ('WindSpeed', 'Enter WindSpeed in m/s: '),
        ('WindDirection', 'Enter WindDirection in degrees (optional, press Enter to skip): ')
    ]
    user_input = {}

    for param, prompt in params:
        while True:
            try:
                value = input(prompt).strip()
                if param == 'WindDirection' and value in ['', 'N/A']:
                    user_input[param] = 180.0
                    break
                user_input[param] = float(value)
                if param in ['PM2.5', 'PM10', 'NO2', 'CO', 'SO2', 'O3', 'Humidity', 'WindSpeed'] and user_input[param] < 0:
                    print(f"{param} cannot be negative. Try again.")
                    continue
                break
            except ValueError:
                if param == 'WindDirection' and value in ['', 'N/A']:
                    user_input[param] = 180.0
                    break
                print(f"Invalid input for {param}. Please enter a number.")

    return user_input

# Main Execution
if __name__ == "__main__":
    # Get user input
    user_input = get_user_input()

    # Compute AQI based on PM2.5
    pm25_base = max(0, user_input['PM2.5'])  # Ensure non-negative
    aqi_label, aqi_value = calculate_aqi(pm25_base)

    # Get health recommendation
    recommendation, emoji = get_health_recommendation(aqi_value)

    # Create 100x80 grid for map with smooth variation
    nx, ny = 100, 80
    x = np.arange(nx)
    y = np.arange(ny)
    X, Y = np.meshgrid(x, y)
    # Add smooth gradient around pm25_base (e.g., ±5 µg/m³)
    C_pm25 = pm25_base + np.sin(X * 0.1) * np.cos(Y * 0.1) * 2.5 + np.random.normal(0, 1, (ny, nx))
    C_pm25 = np.clip(C_pm25, 0, 50)  # Limit to reasonable range

    # Output
    print(f"AQI: {aqi_value} ({aqi_label})")
    print("AQI Ranges:")
    print("0-50: Good")
    print("51-100: Moderate")
    print("101-150: Unhealthy for Sensitive Groups")
    print("151-200: Unhealthy")
    print("201-300: Very Unhealthy")
    print("301-500: Hazardous")
    print(f"{recommendation} {emoji}")

    # Interactive map
    plot_interactive_map(C_pm25, aqi_label, nx, ny, 80, 87.5)

Enter PM2.5 in µg/m³: 40
Enter PM10 in µg/m³: 133
Enter NO2 in ppb: 10
Enter CO in ppm: 670
Enter SO2 in ppb: 4
Enter O3 in ppb: 53
Enter Temperature in Celsius: 36
Enter Humidity in %: 25
Enter WindSpeed in m/s: 4
Enter WindDirection in degrees (optional, press Enter to skip): 180
AQI: 112 (Unhealthy for Sensitive Groups)
AQI Ranges:
0-50: Good
51-100: Moderate
101-150: Unhealthy for Sensitive Groups
151-200: Unhealthy
201-300: Very Unhealthy
301-500: Hazardous
Reduce outdoor activities for sensitive groups. 😷
